In [ ]:
#! pip install -U accelerate
#! pip install -U transformers

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install emoji
import emoji

In [ ]:
train = pd.read_csv("Downloads/train_data.csv")
test = pd.read_csv("Downloads/test_data.csv")
train.head()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
train.groupby('sentiment').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
train = train[~(train['comment_description'].isna())]

In [ ]:
train['comment_description'] = train['comment_description'].apply(lambda x: emoji.demojize(x))

In [ ]:
train['sentiment'].replace({'Negative': 0, 'Positive': 1,'Irrelevant':2}, inplace=True)

In [ ]:
train_texts = train['comment_description'].values.tolist()
train_labels = train['sentiment'].values.tolist()
test_texts = test['comment_description'].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2,random_state=42,stratify=train_labels)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer,TrainingArguments

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=3)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True,return_tensors = 'pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True,return_tensors = 'pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True,return_tensors = 'pt')

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
## Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

In [ ]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentTestDataset(test_encodings)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='sample_data',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True)

In [ ]:
args = TrainingArguments(
    output_dir="sample_data",
    num_train_epochs=5,
    per_device_train_batch_size=8

)

In [ ]:
trainer = Trainer(
    model=model,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=val_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)

In [ ]:
import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
test['sentiment'] = 0 test_texts = test[‘Review’].values.tolist() test_labels = test[‘Sentiment’].values.tolist() 
test_encodings = tokenizer(test_texts, truncation=True, padding=True,return_tensors = ‘pt’).to(“cuda”) 
test_dataset = SentimentDataset(test_encodings, test_labels)
preds = trainer.predict(test_dataset=test_dataset)

In [ ]:
probs = torch.from_numpy(preds[0]).softmax(1)
predictions = probs.numpy()# convert tensors to numpy array
newdf = pd.DataFrame(predictions,columns=['Negative_0','Neutral_1','Positive_2'])
new_df.head()

In [ ]:
def labels(x):
    if x == 0:
        return 'Negative'
    elif x == 1:
        return 'Irrelavant'
    else:
        return 'Positive'

results = np.argmax(predictions,axis=1)
test['sentiment'] = results
test['sentiment'] = test['sentiment'].map(labels)
test.head()